The way I have been implementing distributed slack is unorthodox at best. It is probably also incorrect!

It turns out that $A$ can be simpler than I have been making it. All I need to do is make the angle refernce bus assignment explicit. Then the linear constraints $Ax=b$ become:

\begin{align}
\begin{bmatrix} -I & Y & -k \\ 0 & e_s^\top & 0 \end{bmatrix}
\begin{bmatrix} x \\ \theta \\ \alpha \end{bmatrix} &=
\begin{bmatrix} G_0 + P_0 - D \\ 0 \end{bmatrix}
\end{align}

where I remove columns of $I$ corresponding to non-wind nodes (where there are no deviation variables). Note that the bottom equation is just $e_s^\top \theta = \theta_s = 0$.

My prior confusion was due to my perceived need for an explicit expression of $\alpha$. Without an equation to establish it, I reasoned, $\alpha$ would have no meaning. This is foolish in light of the very nature of the power balance equations: there is no explicit equation for any of the variables, yet every vector in the set $\{x:Ax=b\}$ assigns reasonable values to all of them. We do not solve the power balance equations for any particular variable. We simply write the equations out using the appropriate variables and stack everything up in matrix-vector form. Solutions come out according to linear algebra.

Here is my new method:

In [14]:
function tmp_inst_A(Ridx,T,Y,ref,k)#,tau,line)
    """ Generate the power balance constraint A matrix
    from problem dimensions, admittance matrix,
    and generator participation factors.
    Assumes the admittance matrix is n-by-n.
    
    Returns A, which is (n+1)*T-by-(nr+n+1)*T
    
    * nr is the number of wind farms in the network
    * n is the number of nodes in the network
    * Ridx is a vector indicating wind farm locations
    * T is the number of time steps
    * Y is the admittance matrix (n-by-n)
    * ref is the index of the angle reference bus
    * k is the vector of generator participation factors
    """
    
    function ei(n,i)
        e = zeros(n)
        e[i] = 1.
        return e
    end
    
    n = size(Y,1)
    
    # A has a block diagonal pattern where each
    # block is Atemp:
    Atemp = [[-eye(n)    Y       -k];
            zeros(1,n) ei(n,ref)' 0]
    
    # Remove columns corresponding to non-wind nodes:
    Atemp = sparse(Atemp[:,[Ridx,n+1:2*n+1]])
    
    # Now we can tile the Atemp matrix to generate A:
    A = Atemp
    for t = 2:T
        A = blkdiag(A, Atemp)
    end

    # These rows relate auxiliary angle variables to
    # original angle variables:
#     A2 = tmp_inst_A_scale(n,Ridx,T,tau,ref,line)

#     # Augment A with new rows:
#     A = [[full(A) zeros((n+1)*T,T)]; A2]
    
    return A
end


tmp_inst_A (generic function with 1 method)

In [34]:
A = tmp_inst_A([2],2,[1 2; 3 4],1,[0.3,0.4])
full(A)

6x8 Array{Float64,2}:
  0.0  1.0  2.0  -0.3   0.0  0.0  0.0   0.0
 -1.0  3.0  4.0  -0.4   0.0  0.0  0.0   0.0
  0.0  1.0  0.0   0.0   0.0  0.0  0.0   0.0
  0.0  0.0  0.0   0.0   0.0  1.0  2.0  -0.3
  0.0  0.0  0.0   0.0  -1.0  3.0  4.0  -0.4
  0.0  0.0  0.0   0.0   0.0  1.0  0.0   0.0

Now of course I must change how $b$ is generated. The old way:

In [ ]:
function tmp_inst_b(n,T,G0,P0,D)
    """ Generate the vector b of power balance constraints.
    Assumes G0 and D are nT-by-1 vectors.
    """
    b = FloatingPoint[]
    
    # b = zeros((n+1)*T,1)
    netGen = G0 + P0 - D
    for t = 1:T
        start = (t-1)*n + 1
        stop = start + n - 1
        mismatch = -sum(netGen[start:stop])
        append!(b,netGen[start:stop])
        push!(b,mismatch)
    end

    # Extend b with T additional zeroes:
    tmp_inst_pad_b(b,T)

    return b
end

And the new way, which is a bit simpler:

In [30]:
function tmp_inst_b(n,T,G0,P0,D)
    """ Generate the vector b of power balance constraints.
    Assumes G0 and D are nT-by-1 vectors.
    """
    b = FloatingPoint[]
    netGen = G0 + P0 - D

    for t = 1:T
        start = (t-1)*n + 1
        stop = start + n - 1
        append!(b,netGen[start:stop])
        push!(b,0.)
    end

    # Extend b with T additional zeroes:
    tmp_inst_pad_b(b,T)
    return b
end

tmp_inst_b (generic function with 1 method)

In [32]:
G0 = [1,2,3,4,5,6,7,8,9]
P0 = ones(9)
D = zeros(9)
tmp_inst_b(3,3,G0,P0,D)

12-element Array{FloatingPoint,1}:
  2.0
  3.0
  4.0
  0.0
  5.0
  6.0
  7.0
  0.0
  8.0
  9.0
 10.0
  0.0

Let's split out the augmentation functionality. It doesn't belong in the matrix-building methods. For this I need a new script.